Following: http://www.laurentluce.com/posts/twitter-sentiment-analysis-using-python-and-nltk/
Could do something like https://marcobonzanini.com/2015/05/17/mining-twitter-data-with-python-part-6-sentiment-analysis-basics/ but unsupervised sentiment analysis typically doesn't work well

In [1]:
#Import the csv dataframe
import pandas
from nltk.corpus import stopwords
from nltk.sentiment.util import *

PoGo_labeled = pandas.read_csv('PoGo_Sentiment_Labeled_extended.csv')
PoGo_labeled.drop('Unnamed: 0', axis=1, inplace=True)

/Users/richardknoche/anaconda/lib/python3.5/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
PoGo_labeled.head(n=5)

,latt,location,long,multi-team,screenName,sentiment,text,userId
0,NaN,"Caldwell, ID",NaN,False,desmond_ayala,pos,Which pokemon go team did y'all chose? #valor,2.953472e+09
1,-73.918741,"Brooklyn, NY",40.694338,False,aphrospice,pos,#Magikarp practicing his struggle skills in th...,1.629086e+07
2,NaN,"Bixby, OK",NaN,False,ABellgowan,pos,Pokemon Go is taking over my life #TeamInstinct,1.681036e+09
3,NaN,"Los Angeles, CA",NaN,False,JangoSnow,pos,Go Team Instinct! I like underdogs. :) https:...,1.057434e+07
4,NaN,"Niagara Falls, NY",NaN,False,EmberLighta2,pos,#TeamMystic has total control of Niagara Falls!!,7.513920e+17


In [3]:
PoGo_labeled.ix[:30]

,latt,location,long,multi-team,screenName,sentiment,text,userId
0,NaN,"Caldwell, ID",NaN,False,desmond_ayala,pos,Which pokemon go team did y'all chose? #valor,2.953472e+09
1,-73.918741,"Brooklyn, NY",40.694338,False,aphrospice,pos,#Magikarp practicing his struggle skills in th...,1.629086e+07
2,NaN,"Bixby, OK",NaN,False,ABellgowan,pos,Pokemon Go is taking over my life #TeamInstinct,1.681036e+09
3,NaN,"Los Angeles, CA",NaN,False,JangoSnow,pos,Go Team Instinct! I like underdogs. :) https:...,1.057434e+07
4,NaN,"Niagara Falls, NY",NaN,False,EmberLighta2,pos,#TeamMystic has total control of Niagara Falls!!,7.513920e+17
5,NaN,"Alabama, USA",NaN,False,marshalljansen,pos,Stopped for lunch at a local seafood spot. It...,1.359274e+08
6,NaN,"Columbus, OH",NaN,False,locustXreign,pos,Team valor straightedge fuck everyone else htt...,3.167318e+07
7,NaN,"Lincroft, NJ",NaN,False,ThomasDalziel,pos,I know I'm not the first but I had to! It's to...,5.750384e+08
8,NaN,"Atlanta, GA",NaN,False,NYCcud,pos,Soooo mad I couldn't find this Blastoise on ca...,2.516288e+08
9,NaN,"Virginia Beach, VA",NaN,False,SarahFranchesca,pos,Any team instinct fellas wanna take some gyms ...,5.069618e+07


In [4]:
#Training to identify positive/negative tweets (could also train to identified postive v non-positive (aka neg + nan))
pos_tweets = [(PoGo_labeled.ix[row,'text'],'positive') for row in range(len(PoGo_labeled)) if PoGo_labeled.ix[row,'sentiment'] == 'pos']

neg_tweets = [(PoGo_labeled.ix[row,'text'],'negative') for row in range(len(PoGo_labeled)) if PoGo_labeled.ix[row,'sentiment'] == 'neg']


In [5]:
#Common words that don't help much
from nltk.corpus import stopwords
stopset = set(stopwords.words('english'))

In [6]:
remove_words=['about','are','be','can','do','does','i','my','me','he','she','they','them','you','your','if','on','why','when','are']

for word in remove_words:
    if word in stopset:
        stopset.remove(word)

In [7]:
#If we've made one, import top features list
import pickle
import os.path

use_top_feature=False

if os.path.isfile('top_features.txt'):
    with open('top_features.txt', 'rb') as f:
        use_top_feature_q=input('Use top feature list in classifier? (yes/no) : ')
        if use_top_feature_q == 'yes':
            use_top_feature=True
        top_feature_list = pickle.load(f)

Use top feature list in classifier? (yes/no) : yes


In [8]:
#Using Bigrams
#Combining pos + negative -- filtering out words with <3 letters (keeping things like RT)
#need to run nltk.download() from command line and get stopwords corupus (freezes in the notebook)
import string
from random import shuffle
from nltk.corpus import stopwords
import itertools
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
 
def stopword_filtered_word_feats(words):
    return dict([(word, True) for word in words if word not in stopset])
 
exclude = set(string.punctuation)
excluded_words = ['teammystic','mystic','teamblue','blue',\
                  'teaminstinct','instinct','teamyellow','yellow',\
                  'teamvalor','valor','teamred','red']

#consider removing team references
def filter_tweets(tweets):
    filtered_tweets = []
    for (words, sentiment) in tweets: #equal number neg and pos tweets
        words_filtered=[]
        for word in words.split():
            word = ''.join(ch for ch in word if ch not in exclude) #remove punctuation - improves CV performance a lot
            if len(word) >= 1: #remove one letter words
                #if word not in stopset: #remove common stop words
                    if word[:4] == 'http': #treat URLs the same
                        word='http'
                    if word[0] == '#': #remove hashtags
                        word=word[1:]
                    if (word.lower() not in excluded_words): #remove team identifiers
                        words_filtered.append(word.lower()) #require lower case

        words_filtered = words_filtered #mark_negation(words_filtered)                               
        
        #Can search for bigrams with this
        #bigram_finder = BigramCollocationFinder.from_words(words_filtered)
        #bigrams = bigram_finder.nbest(BigramAssocMeasures.chi_sq, 200)   
        
        # if word not in stopset
        if use_top_feature == True:
            filtered_tweets.append(([word for word in words_filtered if word in top_feature_list],sentiment))
        elif use_top_feature == False:
            filtered_tweets.append(([word for word in words_filtered],sentiment))

    return filtered_tweets

filtered_pos_tweets = filter_tweets(pos_tweets)
filtered_neg_tweets = filter_tweets(neg_tweets)
       

In [9]:
#Divide into training and test datasets - give training set equal number of pos and neg by downsampling positives

total_neg = len(neg_tweets)
total_pos = len(pos_tweets)

'''
#Downsampling the positive tweets
#half the negative tweets go in training
len_train = int(round(len(filtered_neg_tweets)/2)*2)
train_tweets = filtered_neg_tweets[:int(len_train/2)] + filtered_pos_tweets[:int(len_train/2)]

#half of the remaining half go in cv
cv_neg_cutoff = int( (len_train/2) + round((len(filtered_neg_tweets) - len_train/2)/2) )
cv_pos_cutoff = int( (len_train/2) + round((len(filtered_pos_tweets) - len_train/2)/2) )
cv_tweets =  filtered_neg_tweets[int(len_train/2):cv_neg_cutoff] +  filtered_pos_tweets[int(len_train/2):cv_pos_cutoff]  
    
#rest go into testing
test_tweets =  filtered_neg_tweets[cv_neg_cutoff:] +  filtered_pos_tweets[cv_pos_cutoff:]  
'''

'''

#Upsample negative tweets
#half the negative tweets go in training
neg_scale_factor = 3
len_train = int(round(len(filtered_neg_tweets)/2)*2)
train_tweets = filtered_neg_tweets[:int(len_train/2)]*neg_scale_factor + filtered_pos_tweets[:int(neg_scale_factor*len_train/2)]

#half of the remaining half go in cv
cv_neg_cutoff = int( (len_train/2) + round((len(filtered_neg_tweets) - len_train/2)/2) )
cv_pos_cutoff = int( (neg_scale_factor*len_train/2) + round((len(filtered_pos_tweets) - neg_scale_factor*len_train/2)/2) )
cv_tweets =  filtered_neg_tweets[int(len_train/2):cv_neg_cutoff] +  filtered_pos_tweets[int(neg_scale_factor*len_train/2):cv_pos_cutoff]  
    
#rest go into testing
test_tweets =  filtered_neg_tweets[cv_neg_cutoff:] +  filtered_pos_tweets[cv_pos_cutoff:]  


'''

#True ratio of tweets
#half the tweets go into training
len_neg_train = int(round(len(filtered_neg_tweets)*0.5))
len_pos_train = int(round(len(filtered_pos_tweets)*0.5))
train_tweets = filtered_neg_tweets[:int(len_neg_train/2)] + filtered_pos_tweets[:int(len_pos_train/2)]

#half of the remaining half go in cv
cv_neg_cutoff = int( (len_neg_train/2) + round((len(filtered_neg_tweets) - len_neg_train/2)/2) )
cv_pos_cutoff = int( (len_pos_train/2) + round((len(filtered_pos_tweets) - len_pos_train/2)/2) )
cv_tweets =  filtered_neg_tweets[int(len_neg_train/2):cv_neg_cutoff] +  filtered_pos_tweets[int(len_pos_train/2):cv_pos_cutoff]  
    
#rest go into testing
test_tweets =  filtered_neg_tweets[cv_neg_cutoff:] +  filtered_pos_tweets[cv_pos_cutoff:]  


In [10]:
len(train_tweets)

928

In [11]:
train_tweets[1][0]

['team', 'be', 'so', '😂']

In [12]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

#Note: If train_tweets[0] was a list containing unigrams AND bigrams, we could extract the features like so:
#all_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=3)
#bigram_feats = [ feat for feat in all_feats if type(feat) == tuple ]
#unigram_feats = [ feat for feat in all_feats if type(feat) == str ]
#The downside of the method above is you have to have the same top_n and min_freq for bigrams and unigrams

#using stopwords makes pretty much no difference
#best so far: 30,2 500,3: 66% improvement, 80% retention
sentim_analyzer = SentimentAnalyzer()
all_words = sentim_analyzer.all_words([doc for doc in train_tweets])
bigram_feats = sentim_analyzer.bigram_collocation_feats([tweet[0] for tweet in train_tweets],top_n=30,min_freq=2)
unigram_feats = sentim_analyzer.unigram_word_feats(all_words,top_n=300, min_freq=3)

In [13]:
sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)
sentim_analyzer.add_feat_extractor(extract_bigram_feats, bigrams = bigram_feats)

In [14]:
#train_tweets must be a list of labeled tuples with tuples of the form (['word1','word2','word3'],'negative)
#Must be a list even if the list only has one element!
#The list does not need to have the bigrams labeled seperately, but it's okay if they are (they will just be ignored)
#Having ONLY bigram tuples in the word list would not work, since they would all be ignored

training_set = sentim_analyzer.apply_features(train_tweets)
cv_set = sentim_analyzer.apply_features(cv_tweets)
test_set = sentim_analyzer.apply_features(test_tweets)

In [15]:
trainer = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(trainer, training_set)

Training classifier


In [16]:
ev = sentim_analyzer.evaluate(cv_set)
rec_pos = ev['Recall [positive]']
rec_neg = ev['Recall [negative]']

Evaluating NaiveBayesClassifier results...


In [17]:
for key,value in sorted(ev.items()):            
     print('{0}: {1}'.format(key, value))
        
print('Contamination Improvement: ', 100*(1-(1-rec_neg)/rec_pos))

Accuracy: 0.8922413793103449
F-measure [negative]: 0.2574257425742574
F-measure [positive]: 0.9419054996127033
Precision [negative]: 0.29545454545454547
Precision [positive]: 0.9325153374233128
Recall [negative]: 0.22807017543859648
Recall [positive]: 0.9514866979655712
Contamination Improvement:  18.87119113573408


We want high negative recall so that we correctly identify and reject negative tweets often.  Ideally, we would have high negative precision as well.  In this case we have low negaive precision, which means we are identifying a lot of tweets as negative when they are not actually negative.  This is okay as long as it is randomly happening. 
However, if we accidentally identify a positive tweet as negative more often for a specific team, then we are skewing the results for that team. - Throw out any team identifiers in the tweets to solve this problem.

Note: If we had high negative precision we could look at which teams are the most hated by looking at the fraction of negative tweets associated with each tam.  However, since we are randomly throwing tweets into the negative category every now and then, the distribution of hatred should be about 33% for each team -- VERIFY THIS IS THE CASE.

In [ ]:
#Show the 20 most important features
print (classifier.show_most_informative_features(150))

In [ ]:
top_features=classifier.most_informative_features(150)

In [ ]:
#removing contains() wrapping
top_features=[top_features[row][0].split('contains(')[1][:-1] for row in range(len(top_features))]

In [ ]:
#Splitting bigrams into unigrams
cleaned_top_features = []
for row in range(len(top_features)):
    cleaned_top_features += top_features[row].split(' - ')

In [ ]:
import pickle
#Save the top features to text for itterative use

if os.path.isfile('top_features.txt'):
    os.remove('top_features.txt')
with open('top_features.txt', 'wb') as f:
    pickle.dump(cleaned_top_features, f)

In [ ]:
#approximate error improvement
num_states = 50
scale_factor = 5 #scaling from number of labeled data to all data

#old errors
events_per_state = scale_factor*(total_pos+total_neg)/num_states
orig_stat_err = (events_per_state) ** (1/2) / (events_per_state) #sqrt(n)/n gives rough fractional error
orig_sys_err = total_neg/(total_neg+total_pos) 
orig_tot_err = (orig_stat_err ** 2 + orig_sys_err ** 2) ** (1/2)
print ('Original - Stat: ', orig_stat_err, ' Sys: ', orig_sys_err, ' Total: ', orig_tot_err)

#new errors
events_per_state = scale_factor*(total_pos*rec_pos)/num_states
new_stat_err = (events_per_state) ** (1/2) / (events_per_state) #sqrt(n)/n gives rough fractional error
new_sys_err = (total_neg*(1-rec_neg))/(total_neg*(1-rec_neg)+total_pos*rec_pos) 
new_tot_err = (new_stat_err ** 2 + new_sys_err ** 2) ** (1/2)
print ('Improved - Stat: ', new_stat_err, ' Sys: ', new_sys_err, ' Total: ', new_tot_err)

<h1> After optimizing for CV set, get statistics for test set </h1>

In [ ]:
ev = sentim_analyzer.evaluate(test_set)
rec_pos = ev['Recall [positive]']
rec_neg = ev['Recall [negative]']

for key,value in sorted(ev.items()):            
     print('{0}: {1}'.format(key, value))
        
print('Contamination Improvement: ', 100*(1-(1-rec_neg)/rec_pos))